# Model Evaluation

In [1]:
import sys
sys.path.append("../src")

import torch
import numpy as np
import json

from data_loader import FoodDataset
from utils import get_transforms, load_model_and_stats, evaluate_model
from torch.utils.data import Subset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

transform = get_transforms()

Load metadata

In [2]:
with open("../models/model2_metadata.json", "r") as f:
    meta2 = json.load(f)

test_indices = meta2["test_indices"]

Load data (not normalized)

In [3]:
test_dataset = Subset(FoodDataset(
    csv_path = "../data/cleaned-food-data.csv",
    img_dir   = "../data/data/images",
    transform = transform,
    target_mean = None,
    target_std  = None
),
test_indices)

print("Eval dataset size:", len(test_dataset))

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Eval dataset size: 2249


Load model using helper function

In [4]:
model2, mean2, std2 = load_model_and_stats(
    "../models/model_2.pt",
    "../models/model2_metadata.json",
    device=device
)

Evaluate model

In [6]:
results = evaluate_model(model2, test_loader, mean2, std2, device=device)
print("MAE:", results["MAE"])
print("MSE:", results["MSE"])

Evaluating: 100%|██████████| 71/71 [02:24<00:00,  2.03s/it]

MAE: 41.8267585506753
MSE: 9220.5680816474
